In [ ]:
import numpy as np
import scipy.sparse as sp
import multiprocessing

def calcular_probabilidad_paralelo_rho(args):
    inicio, fin, qubit_index, rho_diag = args
    prob = 0.0
    for i in range(inicio, fin): #Recorre N/Hilos veces
        if (i >> qubit_index) & 1: #Operación O(1)
            prob += rho_diag[i].real  # solo parte real. Suma O(1)
    return prob
# Coste temporal por hilo: O(dim/num_hilos)
# Coste espacial: solo variables locales => despreciable


#UTILIZAR TENSOR FLOW. PERO NO LO PIDE, CON QUE FUNCIONE CON LO DE NUMPY Y SCIPY BASTA
# Definimos la clase QRegistryDensity para representar un registro cuántico de n qubits
# y su matriz de densidad asociada. Esta clase permite la manipulación de estados cuánticos
# y la aplicación de puertas cuánticas, así como la medición de qubits y el colapso del estado.
class QRegistryDensity:
    def __init__(self, n, init_state=None, num_hilos=None):
        self.n = n
        self.dim = 2**n
        self.num_hilos = self._ajustar_num_hilos(num_hilos)
        self.pool = multiprocessing.Pool(processes=self.num_hilos)
        self.rho = self._parse_init_state(init_state)
        # Espacio:
        # - dim = 2^n
        # - rho ocupa O(dim^2) complejos 
        # - pool mantiene procesos hijos (memoria separada por proceso ~ligera)
    def _ajustar_num_hilos(self, num_hilos):
        max_hilos = min(multiprocessing.cpu_count(), 2**self.n)
        
        if num_hilos is None:
            num_hilos = max_hilos

        if (2**self.n) // num_hilos <= 3: # Condición para evitar chunks muy pequeños
            num_hilos = max(1, num_hilos // 2)
        return min(num_hilos, max_hilos)
        # Coste tiempo: O(1), solo cálculos enteros
        # Coste espacio: despreciable

    
    def _parse_init_state(self, init_state):
        """ 
        Esta función toma un estado inicial y lo convierte en una matriz de densidad.
        """
        # Dependiendo del tipo de init_state, se inicializa rho de diferentes maneras.
        if init_state is None:
            # Caso 1: Si no se proporciona un estado inicial,
            # se inicializa en el estado puro |0⟩^n como matriz de densidad.
            psi = np.zeros((self.dim, 1), dtype=complex)
            psi[0, 0] = 1
            return psi @ psi.conj().T  # ρ = |ψ⟩⟨ψ|   # Producto matriz-columna x fila => O(dim^2)

        elif isinstance(init_state, list):
            # Caso 2: Si se proporciona una lista, se interpreta como una mezcla estadística
            # de estados puros con sus respectivas probabilidades [(psi1, p1), (psi2, p2), ...]
            rho = np.zeros((self.dim, self.dim), dtype=complex)
            for psi, p in init_state:
                psi = psi.reshape((self.dim, 1))  # aseguramos que psi sea vector columna
                rho += p * (psi @ psi.conj().T)  # suma ponderada de proyecciones
            return rho

        else:
            # Caso 3: init_state es un array de NumPy
            init_state = np.asarray(init_state)
            if init_state.shape == (self.dim,) or init_state.shape == (self.dim, 1):
                # Caso 3a: estado puro como vector
                psi = init_state.reshape((self.dim, 1))
                return psi @ psi.conj().T  # ρ = |ψ⟩⟨ψ|
            elif init_state.shape == (self.dim, self.dim):
                # Caso 3b: init_state ya es una matriz de densidad
                return init_state
            else:
                # Caso no válido: se lanza una excepción
                raise ValueError("init_state inválido.")
            
    def print_state(self):
        """ 
        Esta función imprime la matriz de densidad rho en un formato legible.
        """
        # Se utiliza un formato binario para los índices de la matriz.
        print("Matriz de densidad:")
        for i in range(self.dim):
            for j in range(self.dim):
                print(f"({i:0{self.n}b},{j:0{self.n}b}): {self.rho[i, j]}")
        # Tiempo: O(dim^2) impresiones
        # Espacio: impresión a consola (no memoria en RAM)


    def apply_gate(self, gate, qubits):
        """ 
        Esta función aplica una puerta cuántica a uno o dos qubits del registro cuántico.
        """
        n = self.n
        if isinstance(qubits, int):
            qubits = [qubits]
        qubits = [(n - 1) - q for q in qubits]
        qubits.sort()

        # Construcción del operador extendido
        if len(qubits) == 1: #Puerta de 1 qubit
            target = qubits[0]
            I_left = sp.eye(2**target) if target > 0 else 1
            I_right = sp.eye(2**(n - target - 1)) if (n - target - 1) > 0 else 1
            U = sp.kron(I_left, sp.kron(gate, I_right, format='coo'), format='coo')  # U: sparse matriz de tamaño dim x dim
        elif len(qubits) == 2: #Puerta de 2 qubits
            q1, q2 = qubits
            I_left = sp.eye(2**q1) if q1 > 0 else 1
            I_mid = sp.eye(2**(q2 - q1 - 1)) if (q2 - q1 - 1) > 0 else 1
            I_right = sp.eye(2**(n - q2 - 1)) if (n - q2 - 1) > 0 else 1
            U = sp.kron(I_left, sp.kron(gate, sp.kron(I_mid, I_right), format='coo'), format='coo')  # U: igual pero con un término intermedio
        else:
            raise ValueError("Solo se soportan puertas de 1 o 2 qubits.")

        # Actualizar matriz de densidad: ρ → U ρ U†
        U = U.toarray()  # convierto a densa para producto. # Dense matrix (dim x dim)
        # Espacio: O(dim^2) 
        # Tiempo: O(dim^2) para conversión
        self.rho = U @ self.rho @ U.conj().T
        # Multiplicaciones densas: O(dim^3)
        # Espacio: rho ocupa O(dim^2), se sobrescribe

    def measure_qubit(self, qubit_index):
        """ 
        Esta función mide un qubit específico y devuelve la probabilidad de que esté en el estado |1⟩.
        """
        # La probabilidad se calcula sumando los elementos diagonales de la matriz de densidad
        prob_one = 0.0
        for i in range(self.dim):
            if (i >> qubit_index) & 1:
                prob_one += self.rho[i, i].real
        return float(min(prob_one, 1.0))
        # Tiempo: O(dim)
        # Espacio: solo variable escalar => despreciable
    def measure_paralel_qubit(self, qubit_index):
        """ 
        Esta función mide un qubit específico y devuelve la probabilidad de que esté en el estado |1⟩ utilizando multiprocessing
        """
        chunk_size = self.dim // self.num_hilos
        inicios = [i * chunk_size for i in range(self.num_hilos)]
        finales = [min((i + 1) * chunk_size, self.dim) for i in range(self.num_hilos)]
        diag_rho = np.diag(self.rho)
        args = [(ini, fin, qubit_index, diag_rho) for ini, fin in zip(inicios, finales)]
        resultados = self.pool.map(calcular_probabilidad_paralelo_rho, args)
        return min(sum(resultados), 1.0)

        # Coste temporal:
        # - np.diag -> O(dim)
        # - map con H hilos -> cada uno hace O(dim/H)
        # - Total: O(dim) + O(dim) = O(dim)
        # Coste espacial: diag_rho 

    def _projector(self, qubit_index, value):
        """
        Esta función construye el proyector correspondiente al qubit y valor dados.
        """
        n = self.n
        P0 = np.array([[1, 0], [0, 0]], dtype=complex)
        P1 = np.array([[0, 0], [0, 1]], dtype=complex)
        P = P1 if value == 1 else P0
        qubit = (n - 1) - qubit_index
        I_left = sp.eye(2**qubit) if qubit > 0 else 1
        I_right = sp.eye(2**(n - qubit - 1)) if (n - qubit - 1) > 0 else 1
        # Matrices identidad dispersas (eye) => no se materializan en memoria completa
        full_P = sp.kron(I_left, sp.kron(P, I_right), format='coo')
        return full_P  #.toarray()
        # Coste temporal: O(dim log dim) por la construcción de kron (aproximado)
        # Coste espacial: (crece exponencial con n)


    def collapse(self, qubit_index, value, prob_one):
        """
        Esta función colapsa el estado cuántico en función de la medida realizada.
        """
        # Se utiliza el proyector correspondiente al qubit y valor dados.
        P = self._projector(qubit_index, value)  # Tiempo: O(2^n) si P es dispersa (se construye con kron). 
                                                 # Espacio: O(2^n) elementos no nulos en P (sparse).
        rho_colapsada = P @ self.rho @ P     # Tiempo: O(nnz_P * nnz_rho) → O(1) si ambas son extremadamente dispersas
                                             # Espacio: O(nnz_resultado) → O(1) si la matriz colapsada también es dispersa
        normalizador = prob_one if value == 1 else (1 - prob_one)  # Tiempo: O(1)
                                                                   # Espacio: O(1)

        self.rho = rho_colapsada / normalizador if normalizador > 0 else rho_colapsada  # Tiempo: O(nnz_rho_colapsada) → O(1)
                                                                                        # Espacio: O(nnz) → O(1)
        # Coste temporal: Dominado por la multiplicación de matrices densas => O(dim^3)
        # Coste espacial: Se reescribe rho, sigue siendo la misma dim^2
    def measure(self, qubit_index):
        """ 
        Esta función mide un qubit específico y colapsa el estado cuántico.
        """
        # Se utiliza la función measure_qubit para obtener la probabilidad de |1⟩.
        prob_one = self.measure_qubit(qubit_index)
        value = np.random.choice([0, 1], p=[1 - prob_one, prob_one])
        self.collapse(qubit_index, value, prob_one)
        return value
    def partial_trace(self, qubit_index):
        """
        Calcula la traza parcial de la matriz de densidad sobre todos los qubits
        excepto el indicado (qubit_index). Devuelve una matriz 2x2 (rho_reducida).
        """
        # Convertir a matriz dispersa COO si es necesario
        if isinstance(self.rho, np.ndarray):                   # Tiempo: O(1), Espacio: O(1)
          rho = sp.coo_matrix(self.rho)                      # Tiempo: O(2^2n), Espacio: O(nnz) siendo nnz el numero de entradas no nulas
        else:
          rho = self.rho.tocoo()                             # Tiempo: O(nnz), Espacio: O(nnz)
    
        dim = self.dim  # 2^n                                   # Tiempo: O(1), Espacio: O(1)
        n = self.n                                           # Tiempo: O(1), Espacio: O(1)
    
        reduced = np.zeros((2, 2), dtype=complex)             # Tiempo: O(1), Espacio: O(1)

        for i, j, val in zip(rho.row, rho.col, rho.data):     # Tiempo total: O(nnz)
            # Bit de interés
            bi = (i >> (n - 1 - qubit_index)) & 1             # Tiempo: O(1), Espacio: O(1)
            bj = (j >> (n - 1 - qubit_index)) & 1             # Tiempo: O(1), Espacio: O(1)

            # Eliminamos los demás qubits: verificamos que todos los demás bits son iguales
            mask = ~(1 << (n - 1 - qubit_index))              # Tiempo: O(1), Espacio: O(1)
            if (i & mask) == (j & mask):                      # Tiempo: O(1)
                reduced[bi, bj] += val                        # Tiempo: O(1)

        return reduced                                        # Tiempo: O(1), Espacio: O(1)
        #Coste total en tiempo:  O(nnz) para rho, O(1) para estado puro
        #Coste total en espacio: O(1)

    def concurrence(self, qubit_index):
        """
        Calcula la concurrencia del qubit dado, definida como Tr(rho_reducida^2),
        usando la traza parcial. Funciona tanto para estados puros como para mezclas.
        """
        rho_reducida = self.partial_trace(qubit_index)         # Tiempo: O(nnz), Espacio: O(1)
        pureza = np.trace(rho_reducida @ rho_reducida).real    # Tiempo: O(1), Espacio: O(1)
        return pureza                                          # Tiempo: O(1), Espacio: O(1)
        #Coste total en tiempo:  O(nnz) para rho, O(1) para estado puro
        #Coste total en espacio: O(1)

#IMPORTAMOS LAS PUERTAS QUE HEMOS DEFINIDO EN OTRO ARCHIVO LLAMADO puertas.ipynb
from IPython import get_ipython

get_ipython().run_line_magic("run", "puertas.ipynb")


In [6]:
#Probamos a hacer operaciones

# Crear un registro cuántico de 2 qubits
qr = QRegistryDensity(2, num_hilos=4)
qr.print_state()

# Aplicar una puerta Hadamard al primer qubit
qr.apply_gate(H(), 0)
#qr.print_state()
qr.partial_trace(0) #rho_1 = Tr_2(rho)
# Medir el primer qubit
resultado = qr.measure(0)
print(f"Resultado de la medida del primer qubit: {resultado}")

# Imprimir la matriz de densidad después de la medida
#qr.print_state()

Matriz de densidad:
(00,00): (1+0j)
(00,01): 0j
(00,10): 0j
(00,11): 0j
(01,00): 0j
(01,01): 0j
(01,10): 0j
(01,11): 0j
(10,00): 0j
(10,01): 0j
(10,10): 0j
(10,11): 0j
(11,00): 0j
(11,01): 0j
(11,10): 0j
(11,11): 0j
Resultado de la medida del primer qubit: 0


In [14]:
# Estado mezcla 50% |0>, 50% |1>
psi0 = np.array([1, 0], dtype=complex)
psi1 = np.array([0, 1], dtype=complex)
qr = QRegistryDensity(n=1, init_state=[(psi0, 0.5), (psi1, 0.5)])

qr.print_state()
qr.apply_gate(H(), 0)
qr.print_state()
resultado = qr.measure_qubit(0)
print("Resultado de la medida:", resultado)
qr.print_state()

Matriz de densidad:
(0,0): (0.5+0j)
(0,1): 0j
(1,0): 0j
(1,1): (0.5+0j)
Matriz de densidad:
(0,0): (0.4999999999999999+0j)
(0,1): (-1.1185571585378691e-17+0j)
(1,0): (-1.1185571585378691e-17+0j)
(1,1): (0.4999999999999999+0j)
Resultado de la medida: 0.4999999999999999
Matriz de densidad:
(0,0): (0.4999999999999999+0j)
(0,1): (-1.1185571585378691e-17+0j)
(1,0): (-1.1185571585378691e-17+0j)
(1,1): (0.4999999999999999+0j)
